__Нейросетевая языковая модель на основе LSTM__ 

Евгений Борисов <esborisov@sevsu.ru>

Shivam Bansal   
Language Modelling and Text Generation using LSTMs — Deep Learning for NLP.    
Mar 26, 2018

https://medium.com/@shivambansal36/language-modelling-text-generation-using-lstms-deep-learning-for-nlp-ed36b224b275

In [2]:
# from keras.preprocessing.sequence import pad_sequences
# from keras.layers import Embedding, LSTM, Dense, Dropout
# from keras.preprocessing.text import Tokenizer
# from keras.callbacks import EarlyStopping
# from keras.models import Sequential
# import keras.utils as ku 
# import numpy as np 

In [3]:
import gzip
FILE_DATA = '../data/dostoevsky-besy-p2.txt.gz'
with gzip.open(FILE_DATA,'rt',encoding='utf-8') as f: data = f.read()     
print(len(data))

465595


In [ ]:
# class Tokenizer: Text tokenization utility class.
# Functions
# hashing_trick(...): Converts a text to a sequence of indexes in a fixed-size hashing space.
# one_hot(...): One-hot encodes a text into a list of word indexes of size n.
# text_to_word_sequence(...): Converts a text to a sequence of words (or tokens).
# tokenizer_from_json(...): Parses a JSON tokenizer configuration file and returns a

In [5]:
from random import sample
from nltk.tokenize import sent_tokenize as nltk_sentence_split
# from nltk.tokenize import word_tokenize as nltk_tokenize_word

# text = [ 
#     nltk_tokenize_word(s) # разбиваем предложения на слова
#     for s in nltk_sentence_split(text) # режем текст на отдельные предложения
# ]
# print('предложений: %i\n'%(len(text)))

text = nltk_sentence_split(data)

sample(text,2)

['Что, вы улыбнулись?',
 'Вы никого не оскорбляете, и вас все ненавидят; вы смотрите всем ровней, и вас все боятся, это хорошо.']

In [ ]:
# text.StateBasedSentenceBreaker 
# break_sentences(
#     doc
# )

In [ ]:
# import tensorflow
# tensorflow.__version__

In [6]:
from keras.preprocessing.text import Tokenizer

# # class Tokenizer: Text tokenization utility class.
# # Functions
# # hashing_trick(...): Converts a text to a sequence of indexes in a fixed-size hashing space.
# # one_hot(...): One-hot encodes a text into a list of word indexes of size n.
# # text_to_word_sequence(...): Converts a text to a sequence of words (or tokens).
# # tokenizer_from_json(...): Parses a JSON tokenizer configuration file and returns a

# tokenizer = Tokenizer()

In [7]:
# tokenizer.fit_on_texts(text)

In [29]:
# total_words = len(tokenizer.word_index) + 1

In [18]:
# line = text[1]
# tokenizer.texts_to_sequences([line])[0]


# token_list[:i+1]
# for line in text
# for i,token_idx in enumerate( tokenizer.texts_to_sequences([line])[0] )


In [33]:
# input_sequences = []
# for line in text:
#     token_list = tokenizer.texts_to_sequences([line])[0]
#     for i in range(1, len(token_list)):
#         n_gram_sequence = token_list[:i+1]
#         input_sequences.append(n_gram_sequence)

In [32]:
# import numpy as np
# from keras.preprocessing.sequence import pad_sequences

# max_sequence_len = max([len(x) for x in input_sequences])
# input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
# # input_sequences

In [24]:
 # create predictors and label
# predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

In [30]:
# import keras.utils as ku 
# label = ku.np_utils.to_categorical(label, num_classes=total_words)

In [34]:
def dataset_preparation(data):
    # basic cleanup
    corpus = text = nltk_sentence_split(data)
    
    # data.lower().split("\n")

    # tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1

    # create input sequences using list of tokens
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)

    # pad sequences 
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

    # create predictors and label
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.np_utils.to_categorical(label, num_classes=total_words)

    return predictors, label, max_sequence_len, total_words

In [36]:
# dataset_preparation(data)

In [41]:
# from keras.preprocessing.sequence import pad_sequences
# from keras.preprocessing.text import Tokenizer
# import keras.utils as ku 
# import numpy as np 

from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.models import Sequential

In [42]:
def create_model(predictors, label, max_sequence_len, total_words):
    model = Sequential()
    model.add(Embedding(total_words, 10, input_length=max_sequence_len-1))
    model.add(LSTM(150, return_sequences = True))
    # model.add(Dropout(0.2))
    model.add(LSTM(100))
    model.add(Dense(total_words, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
    model.fit(predictors, label, epochs=100, verbose=1, callbacks=[earlystop])
    print( model.summary() )
    return model 

In [43]:
def generate_text(seed_text, next_words, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

In [44]:
import gzip
FILE_DATA = '../data/dostoevsky-besy-p2.txt.gz'
with gzip.open(FILE_DATA,'rt',encoding='utf-8') as f: data = f.read()     
print(len(data))

465595


In [ ]:
predictors, label, max_sequence_len, total_words = dataset_preparation(data)
model = create_model(predictors, label, max_sequence_len, total_words)
print( generate_text("we naughty", 3, max_sequence_len) )

2022-03-22 14:07:05.598204: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-22 14:07:05.798047: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-22 14:07:05.798235: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-22 14:07:05.799202: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Epoch 1/100


2022-03-22 14:07:14.071789: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8300


2108/2108 [==============================] - 34s 14ms/step - loss: 8.0135 - accuracy: 0.0421
Epoch 2/100
2108/2108 [==============================] - 29s 14ms/step - loss: 7.5218 - accuracy: 0.0448
Epoch 3/100
2108/2108 [==============================] - 29s 14ms/step - loss: 7.3071 - accuracy: 0.0506
Epoch 4/100
2108/2108 [==============================] - 29s 14ms/step - loss: 7.0950 - accuracy: 0.0564
Epoch 5/100
2108/2108 [==============================] - 29s 14ms/step - loss: 6.8832 - accuracy: 0.0621
Epoch 6/100
2108/2108 [==============================] - 29s 14ms/step - loss: 6.6831 - accuracy: 0.0681
Epoch 7/100
2108/2108 [==============================] - 29s 14ms/step - loss: 6.4893 - accuracy: 0.0739
Epoch 8/100
2108/2108 [==============================] - 29s 14ms/step - loss: 6.2848 - accuracy: 0.0791
Epoch 9/100
2108/2108 [==============================] - 29s 14ms/step - loss: 6.0777 - accuracy: 0.0865
Epoch 10/100
2108/2108 [==============================] - 29s 14ms/

2108/2108 [==============================] - 30s 14ms/step - loss: 3.5581 - accuracy: 0.3131
Epoch 27/100
2108/2108 [==============================] - 29s 14ms/step - loss: 3.4650 - accuracy: 0.3290
Epoch 28/100
2108/2108 [==============================] - 30s 14ms/step - loss: 3.3786 - accuracy: 0.3443
Epoch 29/100
2108/2108 [==============================] - 30s 14ms/step - loss: 3.2961 - accuracy: 0.3558
Epoch 30/100
2108/2108 [==============================] - 30s 14ms/step - loss: 3.2166 - accuracy: 0.3694
Epoch 31/100
2108/2108 [==============================] - 30s 14ms/step - loss: 3.1394 - accuracy: 0.3817
Epoch 32/100
2108/2108 [==============================] - 30s 14ms/step - loss: 3.0664 - accuracy: 0.3937
Epoch 33/100
2108/2108 [==============================] - 30s 14ms/step - loss: 2.9958 - accuracy: 0.4043
Epoch 34/100
2108/2108 [==============================] - 30s 14ms/step - loss: 2.9306 - accuracy: 0.4161
Epoch 35/100
2108/2108 [==============================] - 3

2108/2108 [==============================] - 30s 14ms/step - loss: 2.0654 - accuracy: 0.5655
Epoch 53/100
2108/2108 [==============================] - 31s 14ms/step - loss: 2.0266 - accuracy: 0.5733
Epoch 54/100
2108/2108 [==============================] - 29s 14ms/step - loss: 1.9984 - accuracy: 0.5795
Epoch 55/100
2108/2108 [==============================] - 30s 14ms/step - loss: 1.9644 - accuracy: 0.5857
Epoch 56/100
2108/2108 [==============================] - 29s 14ms/step - loss: 1.9315 - accuracy: 0.5924
Epoch 57/100
 430/2108 [=====>........................] - ETA: 24s - loss: 1.7910 - accuracy: 0.6257